<a href="https://colab.research.google.com/github/ddah0329/3th_OUTTA_AI_BootCamp/blob/colab/%EC%9D%B4%EC%BB%A4%EB%A8%B8%EC%8A%A4_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
from bs4 import BeautifulSoup

# 페이지 번호 범위 설정 (1부터 3까지)
page_range = range(1, 4)

# 기본 URL (pageIdx 부분만 동적으로 변경)
base_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100150001&fltDispCatNo=&prdSort=01&pageIdx={}&rowsPerPage=36&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100150001_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%ED%81%AC%EB%A6%BC&smallCategory=%EC%86%8C_%ED%81%AC%EB%A6%BC&checkBrnds=&lastChkBrnd="

# 결과 저장할 리스트
results = []

# 각 페이지의 데이터를 크롤링하는 함수
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 'prd_info' 클래스에 해당하는 div 내부에서 a 태그 추출
    product_infos = soup.select('.prd_info')

    for product in product_infos:
        # 브랜드명과 상품명을 추출
        brand = product.select_one('.tx_brand').get_text(strip=True)
        name = product.select_one('.tx_name').get_text(strip=True)

        # 상품 링크 추출 (href 속성)
        link_tag = product.select_one('a')
        product_link = link_tag['href'] if link_tag else None

        # 추출한 데이터를 results 리스트에 추가
        if product_link:
            results.append({
                "brand": brand,
                "name": name,
                "url": product_link
            })

# 페이지 번호를 변경하면서 크롤링 수행
for page_idx in page_range:
    url = base_url.format(page_idx)
    crawl_page(url)

# 크롤링 결과 출력
for item in results:
    print(f"브랜드: {item['brand']}")
    print(f"상품명: {item['name']}")
    print(f"링크: {item['url']}")
    print("-" * 100)

# 크롤링한 상품의 총 개수 출력
print(f"\n전체상품 : {len(results)}")


브랜드: 토리든
상품명: [8월 올영픽/브랜든콜라보] 토리든 다이브인 수딩크림 더블 한정 기획 (100ml+100ml)
링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000208112&dispCatNo=1000001000100150001&trackingCd=Cat1000001000100150001_Small&t_page=카테고리관&t_click=크림_소_크림__상품상세&t_number=1
----------------------------------------------------------------------------------------------------
브랜드: 에스네이처
상품명: [8월 올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 (60ml+60ml+카밍패드 2매)
링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000202894&dispCatNo=1000001000100150001&trackingCd=Cat1000001000100150001_Small&t_page=카테고리관&t_click=크림_소_크림__상품상세&t_number=2
----------------------------------------------------------------------------------------------------
브랜드: 라로슈포제
상품명: 라로슈포제 시카플라스트 밤 B5+ 100ml 기획 (+3ml 추가증정)
링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000206904&dispCatNo=1000001000100150001&trackingCd=Cat1000001000100150001_Small&t_page=카테고리관&t_click=크림_소_크림__상품상세&